In [1]:
import os
import pandas as pd

In [2]:
os.chdir(r'C:\Users\Komal\Downloads\Datasets')
df=pd.read_csv('spam.csv')
df

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will �_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   v1          5572 non-null   object
 1   v2          5572 non-null   object
 2   Unnamed: 2  50 non-null     object
 3   Unnamed: 3  12 non-null     object
 4   Unnamed: 4  6 non-null      object
dtypes: object(5)
memory usage: 217.8+ KB


In [4]:
df.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis=1,inplace=True)

In [5]:
df=df.rename(columns={'v1':'label','v2':'msg'})

In [6]:
df.head()

,label,msg
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [7]:
df.isnull().any()

label    False
msg      False
dtype: bool

In [8]:
df['label'].value_counts()

ham     4825
spam     747
Name: label, dtype: int64

In [9]:
import nltk
import re
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [10]:
lemmatizer=WordNetLemmatizer()

corpus=[]

for i in range(0,len(df)):
    words=re.sub('[^a-zA-Z]',' ',df['msg'][i])
    words=words.lower()
    words=words.split()
    words=[lemmatizer.lemmatize(word) for word in words if word not in set(stopwords.words('english'))]
    words=' '.join(words)
    corpus.append(words)

In [11]:
corpus[0]

'go jurong point crazy available bugis n great world la e buffet cine got amore wat'

In [12]:
#replacing original msg with transformed msg
df['msg']=corpus

In [13]:
df.head()

,label,msg
0,ham,go jurong point crazy available bugis n great ...
1,ham,ok lar joking wif u oni
2,spam,free entry wkly comp win fa cup final tkts st ...
3,ham,u dun say early hor u c already say
4,ham,nah think go usf life around though


In [14]:
x=df['msg']
y=df['label']

In [15]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer()
x_train_tfidf_vec=tfidf.fit_transform(x_train).toarray()

In [17]:
x_train_tfidf_vec

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [18]:
x_train_tfidf_vec.shape

(4457, 6265)

Multinomial Naive Bayes Classifier

In [19]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB

In [20]:
text_mnb=Pipeline([('tfidf',TfidfVectorizer()),('mnb',MultinomialNB())])

In [21]:
text_mnb.fit(x_train,y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('mnb', MultinomialNB())])

In [22]:
y_pred_mnb=text_mnb.predict(x_test)

In [23]:
y_pred_mnb

array(['ham', 'ham', 'ham', ..., 'ham', 'ham', 'spam'], dtype='<U4')

In [24]:
text_mnb.score(x_train,y_train)

0.979582678932017

In [25]:
text_mnb.score(x_test,y_test)

0.9632286995515695

In [26]:
from sklearn.metrics import confusion_matrix,classification_report

In [27]:
print(confusion_matrix(y_test,y_pred_mnb))

[[965   0]
 [ 41 109]]


In [50]:
print(classification_report(y_test,y_pred_mnb))

              precision    recall  f1-score   support

         ham       0.96      1.00      0.98      1453
        spam       0.99      0.73      0.84       219

    accuracy                           0.96      1672
   macro avg       0.98      0.86      0.91      1672
weighted avg       0.97      0.96      0.96      1672



SVM

In [51]:
from sklearn.svm import LinearSVC
text_svm=Pipeline([('tfidf',TfidfVectorizer()),('svm',LinearSVC())])

In [52]:
text_svm.fit(x_train,y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('svm', LinearSVC())])

In [53]:
y_pred_svm=text_svm.predict(x_test)

In [54]:
y_pred_svm

array(['ham', 'ham', 'spam', ..., 'ham', 'ham', 'ham'], dtype=object)

In [55]:
text_svm.score(x_test,y_test)

0.9826555023923444

In [56]:
print(confusion_matrix(y_test,y_pred_svm))

[[1450    3]
 [  26  193]]


In [57]:
print(classification_report(y_test,y_pred_svm))

              precision    recall  f1-score   support

         ham       0.98      1.00      0.99      1453
        spam       0.98      0.88      0.93       219

    accuracy                           0.98      1672
   macro avg       0.98      0.94      0.96      1672
weighted avg       0.98      0.98      0.98      1672

